In [3]:
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import subprocess
import shutil
from selenium.webdriver.common.by import By
import pandas as pd
import time

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 셀레니움 신버전 크롬 드라이버 패키지

from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.keys import Keys
# 진행률
from tqdm.notebook import tqdm
import re
import os

In [29]:
class find_restaurant_info() :
    def __init__(self , dataframe) :
        self.id_df = dataframe
        self.df = pd.DataFrame(columns = ['식당id' , '사업장명' , '종류' , '주소' , '찾아가는길', '영업시간' 
                                     , '편의' , '설명' ,'전화번호' , '홈페이지', '기타' , '이미지url'])
        
#     def iframe_finder(driver , iframe_id) :
#         wait = WebDriverWait(driver, 10)
#         driver.switch_to.default_content()
#         # self.iframe
#         iframe = wait.until(EC.presence_of_element_located((By.ID, iframe_id)))
#         driver.switch_to.frame(iframe)
        
    def rest_info(self , rest_name , rest_id) :    
        append_df = pd.DataFrame(columns = ['식당id' , '사업장명' , '종류' , '주소' , '찾아가는길', '영업시간' 
                                     , '편의' , '설명' ,'전화번호' , '홈페이지', '기타' , '이미지url'])
        name = rest_name
        id_num = rest_id
        
        append_df['식당id'] = [rest_id]
        append_df['사업장명'] = [rest_name]
        
        # Chrome headless 설정
        options = Options()
        # options.add_argument('--headless')

        # 드라이버 생성
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

        # name = '고기다 성산흑돼지'
        # 리뷰 페이지 열기
        print(f'현재 크롤링이 진행중인 검색어 : {name}')

        url = f"https://map.naver.com/v5/search/{name}/place/{id_num}"

        driver.get(url)
        wait = WebDriverWait(driver, 5)
        iframe = wait.until(EC.presence_of_element_located((By.ID, "entryIframe")))
        driver.switch_to.frame(iframe)

        #iframe_finder('entryIframe')

        try :
            wait.until(EC.presence_of_element_located((By.CLASS_NAME, "rvCSr")))
            time.sleep(0.2)
            show_rest_info_click = driver.find_elements(by = By.CLASS_NAME , value ='rvCSr')
            if show_rest_info_click is not None :
                for click in show_rest_info_click :
                    click.click()
                    scarp_rest_info = driver.find_elements(by = By.CLASS_NAME , value = 'PIbes > div')
        except :
            print('펼칠 내용이 없습니다.')
            pass

        # result = re.sub(pattern, "", text_2)
        n = 0
        # lst = []
        
        pattern1 = r'(접기|오늘 휴무|펼쳐보기|복사|내용 더보기|영업 중|안내\n복사|영업 종료|이용시간을 알려주세요.)'
        pattern1_plus = r'(정보 수정 제안\n정보 수정 제안하기|사장님, 플레이스를\n무료로 직접 관리하세요!\n권한 받기|영업시간 정보 수정\n|\n추가)'
        pattern2 = r'\n(2,)'
        pattern3 = r'\n\n'
        
        
        # 이미지 url 가져오기
        try :
            print('이미지 url 가져오는중')
            wait.until(EC.presence_of_element_located((By.CLASS_NAME, "fNygA")))
            time.sleep(0.3)

            rest_img = driver.find_elements(by = By.CLASS_NAME , value ='fNygA > a > div')
            rest_img_urls = []
            for stylejs in rest_img :

                url_test_data = stylejs.get_attribute('style')
                url_find_num = url_test_data.find('url("')
                if 'jpg' in url_test_data :
                    url_find_num_end = url_test_data.find('jpg')
                else :
                    url_find_num_end = url_test_data.find('.")')
                rest_img_url = url_test_data[url_find_num + 5 : url_find_num_end+3]
                if rest_img_url != '' :
                    rest_img_urls.append(rest_img_url)
            append_df['이미지url'] = [rest_img_urls]
            # image_url = rest_img_urls
        except : 
            # 추가 식당 정보 이미지가 한 개일 경우 위 코드 복붙
            try :
                wait.until(EC.presence_of_element_located((By.CLASS_NAME, "K0PDV")))
                time.sleep(0.3)

                rest_img = driver.find_elements(by = By.CLASS_NAME , value ='K0PDV')
                rest_img_urls = []
                for stylejs in rest_img :

                    url_test_data = stylejs.get_attribute('style')
                    url_find_num = url_test_data.find('url("')
                    if 'jpg' in url_test_data :
                        url_find_num_end = url_test_data.find('jpg')
                    else :
                        url_find_num_end = url_test_data.find('.")')
                    rest_img_url = url_test_data[url_find_num + 5 : url_find_num_end+3]
                    if rest_img_url != '' :
                        rest_img_urls.append(rest_img_url)
                append_df['이미지url'] = [rest_img_urls]
                # image_url = rest_img_urls
            except : 
                print('No restaurant image')
                append_df['이미지url'] = ''
                # image_url=''
                pass


        print('이미지 url 가져오기 종료')

        # 식당 종류 가져오기
        
        try :
            print('식당의 종류를 가져오는 중입니다.')
            wait.until(EC.presence_of_element_located((By.CLASS_NAME , "DJJvD")))
            time.sleep(0.2)
            rest_kind = driver.find_element(by=By.CLASS_NAME , value = "DJJvD").text
            kind = rest_kind
            append_df['종류'] = kind
            print('식당 종류 : ' , kind)
            
        except :
            print('No restaurant kind')
            kind = ''
        
        print('식당의 종류 가져오기 종료')
        
#         address = ''
#         where = ''
#         tel = ''
#         convi = ''
#         explain = ''
#         other = ''
#         opcd = ''
        
            
            
        try :
            print('식당 정보를 가져오는 중입니다.')
            wait.until(EC.presence_of_element_located((By.CLASS_NAME, "PIbes")))
            time.sleep(0.2)
            review_information = driver.find_elements(by = By.CLASS_NAME , value = 'PIbes > div')
            # 식당 정보 가져오기
            for t in review_information :
                # print(t.text)
                # print(n)
                # 정규표현식으로 데이터 정리
                text = re.sub(pattern1, "", t.text)
                text = re.sub(pattern1_plus, "", text)
                text = re.sub(pattern2, "", text)
                text = re.sub(pattern3, "\n", text)
                text = text.rstrip('\n')
                # print(text)
                # print()
                n += 1
                print('text ' , text)
                if text != '' :
                    if '주소' in text :
                        text = re.sub('주소\n', "", text)
                        append_df['주소'] = [text]
                        # address = text
                    elif '찾아가는길' in text :
                        text = re.sub('찾아가는길\n', "", text)
                        append_df['찾아가는길'] = [text]
                        # where = text
                    elif '영업시간' in text :
                        text = re.sub('영업시간\n', "", text)
                        append_df['영업시간'] = [text]
                        # opcd = text
                    elif '064' in text :
                        text = re.sub('전화번호\n', "", text)
                        append_df['전화번호'] = [text]
                        # tel = text
                    elif '050' in text :
                        text = re.sub('전화번호\n', "", text)
                        append_df['전화번호'] = [text]
                        # tel = text
                    elif '편의' in text :
                        text = re.sub('편의\n', "", text)
                        append_df['편의'] = [text]
                        # convi = text
                    elif '설명' in text :
                        text = re.sub('설명\n', "", text)
                        append_df['설명'] = [text]
                        # explain = text
                    elif '홈페이지' in text :
                        text = re.sub('홈페이지\n' , "" , text)
                        append_df['홈페이지'] = [text]
                    else :
                        append_df['기타'] = [text]
                        # other = text
                else : 
                    pass
                # lst.append(text)
        except :
            print('식당 정보를 가져오지 못했습니다.')
            pass
        print('식당 정보 가져오기 종료')
        # 식당 메뉴 가져오기
        
        menu_lst = []
        
        try :
            print('식당 메뉴를 가져오는 중')
            time.sleep(0.3)
            wait.until(EC.presence_of_element_located((By.CLASS_NAME, "veBoZ")))
            category = driver.find_elements(by = By.CLASS_NAME , value = 'veBoZ')
            tlst = [c.text for c in category]
            if '메뉴' not in tlst :
                print('no_menu')
                pass
            else :
                time.sleep(0.2)
                for num , tag in enumerate(category) :
                    # print(tag.text)
                    if tag.text == '메뉴' :
                        driver.find_elements(by = By.CLASS_NAME
                                            , value = 'veBoZ')[num].click()
                        try :
                            wait.until(EC.presence_of_element_located((By.CLASS_NAME, "lPzHi")))
                            menus = driver.find_elements(by=By.CLASS_NAME 
                                              , value = 'lPzHi')
                            print('메뉴 목록 저장중')
                            time.sleep(1)
                            for menu in menus :
                                menu_lst.append(menu.text)
                            append_df['메뉴'] = [menu_lst]
                            print('메뉴 목록 저장완료')
                        except :
                            try :
                                wait.until(EC.presence_of_element_located((By.CLASS_NAME, "tit")))
                                menus = driver.find_elements(by=By.CLASS_NAME 
                                                  , value = 'tit')
                                print('메뉴 목록 저장중')
                                time.sleep(1)
                                for menu in menus :
                                    menu_lst.append(menu.text)
                                append_df['메뉴'] = [menu_lst]
                                print('메뉴 목록 저장완료')
                            except :
                                print('메뉴를 찾지 못하였습니다.')
                                pass
                        
                    else :
                        print(tag.text)
                        pass
        except :
            pass
        
        print('식당 메뉴 가져오기 종료')
        driver.quit()
        print('크롤링 종료')
        print()
        
        concat_df = pd.concat([self.df , append_df] , axis = 0)
        self.df = concat_df
        return self.df
    
        
    def crawling_info(self) :
        rest_names = self.id_df.사업장명.tolist()
        rest_ides = self.id_df.식당id.tolist()
        region_01 = self.id_df.시도.loc[0]
        region_02 = self.id_df.시군구.loc[0]
        region_03 = self.id_df.읍면동.loc[0]
        
        if not os.path.exists(f'{region_01}') :
            os.makedirs(f'{region_01}') 
        else :
            pass
        if not os.path.exists(f'{region_01}/{region_02}') :
            os.makedirs(f'{region_01}/{region_02}') 
        else :
            pass
        if not os.path.exists(f'{region_01}/{region_02}/식당정보') :
            os.makedirs(f'{region_01}/{region_02}/식당정보') 
        else :
            pass
        count = 1
        
        for name , rest_id in tqdm(zip(rest_names , rest_ides)) :
            print(count)
            self.rest_info(name , rest_id)
            count += 1
            
        merged_df = self.df.merge(self.id_df, on=['사업장명','식당id'])
#         merged_frm = pd.concat([self.df,self.id_df] , axis=1)
        merged_df.to_excel(f'{region_01}/{region_02}/식당정보/{region_01}_{region_02}_{region_03}_식당정보_info.xlsx', index=False )
        return merged_df

In [33]:
data = pd.read_csv('제주특별자치도/제주시/식당정보/제주특별자치도_제주시_연동_식당id.csv' , encoding='euc-kr')

In [34]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 968 entries, 0 to 967
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   사업장명        968 non-null    object 
 1   식당id        968 non-null    int64  
 2   업종구분소분류     968 non-null    object 
 3   상세영업상태명     968 non-null    object 
 4   소재지전체주소     968 non-null    object 
 5   도로명전체주소     968 non-null    object 
 6   도로명우편번호     968 non-null    float64
 7   시도          968 non-null    object 
 8   시도_시군구      968 non-null    object 
 9   시군구         968 non-null    object 
 10  시도_시군구_읍면동  968 non-null    object 
 11  읍면동         968 non-null    object 
 12  시도코드        968 non-null    int64  
 13  시군구코드       968 non-null    int64  
 14  읍면동코드       968 non-null    int64  
dtypes: float64(1), int64(4), object(10)
memory usage: 113.6+ KB


In [20]:
test01 = find_restaurant_info(data)

In [21]:
dataframe_01 = test01.crawling_info()

0it [00:00, ?it/s]

1
현재 크롤링이 진행중인 검색어 : 봉플라봉뱅
이미지 url 가져오는중
이미지 url 가져오기 종료
식당의 종류를 가져오는 중입니다.
식당 종류 :  양식
식당의 종류 가져오기 종료
식당 정보를 가져오는 중입니다.
text  주소
제주 제주시 문송1길 6-1 1층 봉플라봉뱅
text  찾아가는길
제주도청 주차장 뒷골목
text  영업시간
17:30에 영업 시작
17시 30분에 영업 시작
text  0507-1408-0417
text  결제수단
지역화폐(카드), 제로페이
text  편의
단체석, 주차, 예약, 무선 인터넷
text  홈페이지
https://app.catchtable.co.kr/ct/shop/bonplatbonvin
인스타그램
text  설명
Lunch 12:00 ~ 14:30 (목, 금, 토)
Dinner 17:30 ~ 23:00
Last Order 22:00
봉플라봉뱅은 좋은 음식과 좋은 와인이라는 뜻으로 셰프와 소믈리에가 운영하는 공간입니다.
봉플라봉뱅이 좋은 추억으로 남았으면 좋겠습니다.
text  세스코 멤버스
2023.05.
2024년08월
 최근 통합방제 점검월
text  
식당 정보 가져오기 종료
식당 메뉴를 가져오는 중
홈
소식
메뉴 목록 저장중
메뉴 목록 저장완료
예약
리뷰
사진
식당 메뉴 가져오기 종료
크롤링 종료

2
현재 크롤링이 진행중인 검색어 : 스톤아일랜드탭하우스
이미지 url 가져오는중
이미지 url 가져오기 종료
식당의 종류를 가져오는 중입니다.
식당 종류 :  맥주,호프
식당의 종류 가져오기 종료
식당 정보를 가져오는 중입니다.
text  주소
제주 제주시 삼무로7길 16 1층 스톤아일랜드탭하우스
text  영업시간
17:00에 영업 시작
17시 0분에 영업 시작
text  0507-1410-9009
text  가격 정보 수정 제안
가격표 사진을 올려주세요.
text  결제수단
제로페이
text  편의
단체석, 주차, 포장, 예약, 무선 인터넷, 남/녀 화장실 구분
text  홈페이지
http://instagra

In [27]:
dataframe_01

,식당id_x,사업장명,종류,주소,찾아가는길,영업시간,편의,설명,전화번호,홈페이지,...,도로명전체주소,도로명우편번호,시도,시도_시군구,시군구,시도_시군구_읍면동,읍면동,시도코드,시군구코드,읍면동코드
0,1713847517,봉플라봉뱅,양식,제주 제주시 문송1길 6-1 1층 봉플라봉뱅,제주도청 주차장 뒷골목,17:30에 영업 시작\n17시 30분에 영업 시작,"단체석, 주차, 예약, 무선 인터넷","Lunch 12:00 ~ 14:30 (목, 금, 토)\nDinner 17:30 ~ ...",0507-1408-0417,https://app.catchtable.co.kr/ct/shop/bonplatbo...,...,"제주특별자치도 제주시 문송1길 6-1, 정인하우스 1층 (연동)",63122.0,제주특별자치도,제주특별자치도 제주시,제주시,제주특별자치도 제주시 연동,연동,5000000000,5011000000,5011013700
1,1244422132,스톤아일랜드탭하우스,"맥주,호프",제주 제주시 삼무로7길 16 1층 스톤아일랜드탭하우스,NaN,17:00에 영업 시작\n17시 0분에 영업 시작,"단체석, 주차, 포장, 예약, 무선 인터넷, 남/녀 화장실 구분",저희 스톤아일랜드탭하우스는 제주최초로 셀프시스템을 도입한 스마트펍으로 20가지 수제...,0507-1410-9009,http://instagram.com/stoneisland_taphouse,...,"제주특별자치도 제주시 삼무로7길 16, 1층 (연동)",63118.0,제주특별자치도,제주특별자치도 제주시,제주시,제주특별자치도 제주시 연동,연동,5000000000,5011000000,5011013700
2,1482063535,연화양꼬치,양꼬치,제주 제주시 연동8길 28,NaN,NaN,NaN,1년미만 부드러운 양고기를 준비하고 있습니다,064-749-1001,NaN,...,"제주특별자치도 제주시 연동8길 28, 1층 105,106호 (연동)",63136.0,제주특별자치도,제주특별자치도 제주시,제주시,제주특별자치도 제주시 연동,연동,5000000000,5011000000,5011013700
3,1028483996,24시누름돌김치찌개,한식,제주 제주시 노연로 146,NaN,영업 전\n08:00에 영업 시작\n8시 0분에 영업 시작,NaN,NaN,064-742-7076,NaN,...,"제주특별자치도 제주시 노연로 146, 1층 (연동)",63133.0,제주특별자치도,제주특별자치도 제주시,제주시,제주특별자치도 제주시 연동,연동,5000000000,5011000000,5011013700
4,1109507832,영화루,중식당,제주 제주시 사장길 15-1,NaN,10:00에 영업 시작\n10시 0분에 영업 시작,NaN,NaN,064-742-8285,NaN,...,"제주특별자치도 제주시 사장길 15-1, 1층 (연동)",63133.0,제주특별자치도,제주특별자치도 제주시,제주시,제주특별자치도 제주시 연동,연동,5000000000,5011000000,5011013700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
975,1318887932,크라우드피씨방제주삼무점,PC방,제주 제주시 신광로 16,NaN,NaN,NaN,NaN,NaN,NaN,...,"제주특별자치도 제주시 신광로 16, 1층 101호 (연동)",63125.0,제주특별자치도,제주특별자치도 제주시,제주시,제주특별자치도 제주시 연동,연동,5000000000,5011000000,5011013700
976,1880272443,만가복,"슈퍼,마트",제주 제주시 연동1길 35,NaN,NaN,NaN,NaN,NaN,NaN,...,"제주특별자치도 제주시 연동1길 35, 1층 (연동)",63131.0,제주특별자치도,제주특별자치도 제주시,제주시,제주특별자치도 제주시 연동,연동,5000000000,5011000000,5011013700
977,1778945379,토라네코제주본점,케이크전문,제주 제주시 삼무로11길 9 남해오네뜨 상가 103호,NaN,영업 전\n10:00에 영업 시작\n10시 0분에 영업 시작,NaN,"토라네코 제주본점입니다! 꾹꾹이 치즈케이크와 치즈브륄레, 얼그레이 브륄레를 판매합니...",0507-1355-9227,https://www.instagram.com/jejutoraneko_official,...,"제주특별자치도 제주시 삼무로11길 9, 1층 103호 (연동, 남해오네뜨)",63118.0,제주특별자치도,제주특별자치도 제주시,제주시,제주특별자치도 제주시 연동,연동,5000000000,5011000000,5011013700
978,1887309829,귀아랑2길베이커리카페,베이커리,제주 제주시 귀아랑2길 8 1층,제주시 연동 신제주로터리 제주은행 뒷골목,영업 전\n09:00에 영업 시작\n에 영업 시작,"포장, 무선 인터넷",베이커리카페,0507-1335-1972,https://www.instagram.com/gwia_rang2gilbakeryc...,...,"제주특별자치도 제주시 귀아랑2길 8, 1층 (연동)",63119.0,제주특별자치도,제주특별자치도 제주시,제주시,제주특별자치도 제주시 연동,연동,5000000000,5011000000,5011013700


In [30]:
test02 = find_restaurant_info(data[:3])

In [31]:
dataframe_02 = test02.crawling_info()

0it [00:00, ?it/s]

1
현재 크롤링이 진행중인 검색어 : 봉플라봉뱅
이미지 url 가져오는중
이미지 url 가져오기 종료
식당의 종류를 가져오는 중입니다.
식당 종류 :  양식
식당의 종류 가져오기 종료
식당 정보를 가져오는 중입니다.
text  주소
제주 제주시 문송1길 6-1 1층 봉플라봉뱅
text  찾아가는길
제주도청 주차장 뒷골목
text  영업시간
17:30에 영업 시작
17시 30분에 영업 시작
text  0507-1408-0417
text  결제수단
지역화폐(카드), 제로페이
text  편의
단체석, 주차, 예약, 무선 인터넷
text  홈페이지
https://app.catchtable.co.kr/ct/shop/bonplatbonvin
인스타그램
text  설명
Lunch 12:00 ~ 14:30 (목, 금, 토)
Dinner 17:30 ~ 23:00
Last Order 22:00
봉플라봉뱅은 좋은 음식과 좋은 와인이라는 뜻으로 셰프와 소믈리에가 운영하는 공간입니다.
봉플라봉뱅이 좋은 추억으로 남았으면 좋겠습니다.
text  세스코 멤버스
2023.05.
2024년08월
 최근 통합방제 점검월
text  
식당 정보 가져오기 종료
식당 메뉴를 가져오는 중
홈
소식
메뉴 목록 저장중
메뉴 목록 저장완료
예약
리뷰
사진
식당 메뉴 가져오기 종료
크롤링 종료

2
현재 크롤링이 진행중인 검색어 : 스톤아일랜드탭하우스
이미지 url 가져오는중
이미지 url 가져오기 종료
식당의 종류를 가져오는 중입니다.
식당 종류 :  맥주,호프
식당의 종류 가져오기 종료
식당 정보를 가져오는 중입니다.
text  주소
제주 제주시 삼무로7길 16 1층 스톤아일랜드탭하우스
text  영업시간
17:00에 영업 시작
17시 0분에 영업 시작
text  0507-1410-9009
text  가격 정보 수정 제안
가격표 사진을 올려주세요.
text  결제수단
제로페이
text  편의
단체석, 주차, 포장, 예약, 무선 인터넷, 남/녀 화장실 구분
text  홈페이지
http://instagra

In [32]:
dataframe_02

,식당id,사업장명,종류,주소,찾아가는길,영업시간,편의,설명,전화번호,홈페이지,...,도로명전체주소,도로명우편번호,시도,시도_시군구,시군구,시도_시군구_읍면동,읍면동,시도코드,시군구코드,읍면동코드
0,1713847517,봉플라봉뱅,양식,제주 제주시 문송1길 6-1 1층 봉플라봉뱅,제주도청 주차장 뒷골목,17:30에 영업 시작\n17시 30분에 영업 시작,"단체석, 주차, 예약, 무선 인터넷","Lunch 12:00 ~ 14:30 (목, 금, 토)\nDinner 17:30 ~ ...",0507-1408-0417,https://app.catchtable.co.kr/ct/shop/bonplatbo...,...,"제주특별자치도 제주시 문송1길 6-1, 정인하우스 1층 (연동)",63122.0,제주특별자치도,제주특별자치도 제주시,제주시,제주특별자치도 제주시 연동,연동,5000000000,5011000000,5011013700
1,1244422132,스톤아일랜드탭하우스,"맥주,호프",제주 제주시 삼무로7길 16 1층 스톤아일랜드탭하우스,NaN,17:00에 영업 시작\n17시 0분에 영업 시작,"단체석, 주차, 포장, 예약, 무선 인터넷, 남/녀 화장실 구분",저희 스톤아일랜드탭하우스는 제주최초로 셀프시스템을 도입한 스마트펍으로 20가지 수제...,0507-1410-9009,http://instagram.com/stoneisland_taphouse,...,"제주특별자치도 제주시 삼무로7길 16, 1층 (연동)",63118.0,제주특별자치도,제주특별자치도 제주시,제주시,제주특별자치도 제주시 연동,연동,5000000000,5011000000,5011013700
2,1482063535,연화양꼬치,양꼬치,제주 제주시 연동8길 28,NaN,NaN,NaN,1년미만 부드러운 양고기를 준비하고 있습니다,064-749-1001,NaN,...,"제주특별자치도 제주시 연동8길 28, 1층 105,106호 (연동)",63136.0,제주특별자치도,제주특별자치도 제주시,제주시,제주특별자치도 제주시 연동,연동,5000000000,5011000000,5011013700


In [37]:
식당정보 = pd.read_excel('제주특별자치도/제주시/식당정보/제주특별자치도_제주시_연동_식당정보_info.xlsx')
식당정보.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 980 entries, 0 to 979
Data columns (total 26 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   식당id        980 non-null    int64 
 1   사업장명        980 non-null    object
 2   종류          979 non-null    object
 3   주소          980 non-null    object
 4   찾아가는길       314 non-null    object
 5   영업시간        746 non-null    object
 6   편의          528 non-null    object
 7   설명          537 non-null    object
 8   전화번호        858 non-null    object
 9   홈페이지        300 non-null    object
 10  기타          677 non-null    object
 11  이미지url      980 non-null    object
 12  메뉴          776 non-null    object
 13  업종구분소분류     980 non-null    object
 14  상세영업상태명     980 non-null    object
 15  소재지전체주소     980 non-null    object
 16  도로명전체주소     980 non-null    object
 17  도로명우편번호     980 non-null    int64 
 18  시도          980 non-null    object
 19  시도_시군구      980 non-null    object
 20  시군구       

In [39]:
식당리뷰 = pd.read_excel('제주특별자치도/제주시/연동/치킨신드롬신제주점_review.xlsx')
식당리뷰.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   리뷰        32 non-null     object
 1   리뷰이미지url  10 non-null     object
 2   작성자프로필    33 non-null     object
 3   My플레이스    33 non-null     object
 4   작성자이름     33 non-null     object
 5   리뷰작성수     33 non-null     object
 6   리뷰작성일     33 non-null     object
 7   식당방문수     33 non-null     object
 8   리뷰인증방식    33 non-null     object
 9   식당id      33 non-null     int64 
 10  사업장명      33 non-null     object
dtypes: int64(1), object(10)
memory usage: 3.0+ KB
